In [7]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from collections import defaultdict

### Convert Dataset Name

In [8]:
def separate_name(dataset_name):
    a = re.split('/', dataset_name)
    a = re.findall(r"[\w']+|[.,!?;]", a[-1])
    a = a[: -2]
    c = []
    for i in a:
        c += re.split("_", i)
    dataset_name = " ".join(c)
    return dataset_name.lower()

In [9]:
meta_df = pd.DataFrame({'Name':[], 'Cols':[]})
i = 1
for root, dirs, files in os.walk("dataset/", topdown=False):
    for name in files:
        path = os.path.join(root, name)
        ds_name = path
        ds_name = separate_name(ds_name)
        ds_cols = list(pd.read_csv(path).columns)
        ds_cols = [t.lower() for t in ds_cols]
        meta_df = meta_df.append({'Name':ds_name, 'Cols':ds_cols}, ignore_index=True)
        i = i + 1

In [85]:
def metadata_df(data_location):
    meta_df = pd.DataFrame({'Name':[], 'Cols':[]})
    i = 1
    for root, dirs, files in os.walk(data_location, topdown=False):
        for name in files:
            path = os.path.join(root, name)
            ds_name = path
            ds_name = separate_name(ds_name)
            ds_cols = list(pd.read_csv(path).columns)
            ds_cols = [t.lower() for t in ds_cols]
            meta_df = meta_df.append({'Name':ds_name, 'Cols':ds_cols, 'File Name': path}, ignore_index=True)
            i = i + 1
    return meta_df

In [87]:
meta_df = metadata_df('dataset/')

In [108]:
meta_df

,Cols,Name,File Name
0,[unnamed: 0],,dataset/.DS_Store
1,"[dbn, school name, num of sat test takers, sat...",2012 sat results,dataset/2012_SAT_Results.csv
2,"[vendor_formal_name, vendor_dba, contact_name,...",m wbe lbe and ebe certified business list,dataset/M_WBE__LBE__and_EBE_Certified_Business...
3,"[cnbio_org_name, number and street, cnadrprf_a...",lower manhattan retailers,dataset/Lower_Manhattan_Retailers.csv
4,"[dbn, name, schoolyear, fl_percent, frl_percen...",2006 2012 school demographics and accountabi...,dataset/2006_-_2012_School_Demographics_and_Ac...
5,"[company name, subindustry, sub subindustry, a...",times square entertainment venues,dataset/Times_Square_Entertainment_Venues.csv
6,"[project name, project description, stimulus t...",federal stimulus data,dataset/Federal_Stimulus_Data.csv
7,"[jurisdiction name, count participants, count ...",school district breakdowns,dataset/School_District_Breakdowns.csv
8,"[dca license number, license type, license exp...",legally operating businesses,dataset/Legally_Operating_Businesses.csv
9,"[demographic, borough, cohort, total cohort, t...",2005 2010 graduation outcomes by borough,dataset/2005-2010__Graduation_Outcomes_-__By_B...


### TFIDF

In [98]:
def search_dataset_sim(kw):
    key_word = kw
    meta_df_tfidf = meta_df.copy()
    meta_df_tfidf = meta_df_tfidf.append({'Name':key_word, 'Cols':"TBD"}, ignore_index=True)
    # meta_df_tfidf["Name"] = ["".join(t) for t in meta_df_tfidf["Name"]]

    tfidf_vec = TfidfVectorizer(stop_words='english',ngram_range=(1,2))
    cols_tfidf = tfidf_vec.fit_transform(meta_df_tfidf['Name'])
    dense_mtx = cols_tfidf.todense()
    v_k = cols_tfidf[-1,:].todense()
    sim = []
    for i in range(dense_mtx.shape[0] - 1):
        v_i = cols_tfidf[i,:].todense()
        sim.append(np.dot(v_i, v_k.T)[0,0])
    indexs = np.argsort(sim)[-5:][::-1]
    return meta_df_tfidf['File Name'][indexs]

In [101]:
search_dataset_sim("school")

7                dataset/School_District_Breakdowns.csv
4     dataset/2006_-_2012_School_Demographics_and_Ac...
10    dataset/2010_-_2011_School_Attendance_and_Enro...
9     dataset/2005-2010__Graduation_Outcomes_-__By_B...
8              dataset/Legally_Operating_Businesses.csv
Name: File Name, dtype: object

### Inv Index

In [112]:
def column_inverted(DataFrame):
    dict_col_inverted = defaultdict(list)
    for ind, i in enumerate(meta_df["Cols"]):
        for j in i:
            if j not in dict_col_inverted.keys():
                dict_col_inverted[j] = meta_df["File Name"][ind]
            else:
                dict_col_inverted[j]+=(meta_df["File Name"][ind])
    return dict_col_inverted

In [113]:
dict_col_inverted = column_inverted(meta_df)

In [114]:
def search_column_exa():
    column_name = input("Please enter column name: ")
    if column_name in dict_col_inverted:
        datasets = dict_col_inverted[column_name]
        return(datasets)
    else:
        return("Sorry, we can't find this column name in any dataset.")

In [116]:
search_column_exa()

Please enter column name: schoolyear


'dataset/2006_-_2012_School_Demographics_and_Accountability_Snapshot.csv'